In [117]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from pyspark.sql import SparkSession
from bookutils import *

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [33]:
k = pd.read_csv('./data/bookListSearch.csv',encoding='cp949',index_col=0)

k.head(1)

,도서명,저자,출판사,ISBN,주제분류번호,등록일자
0,(그림과 실습으로 배우는) 도커 & 쿠버네티스:개념과 작동 원리가 쏙쏙 이해되는 완...,오가사와라 시게타카 지음 ;심효섭 옮김,위키북스,9791158393038,005.1,2022-06-22


In [111]:
def loadLibBook(libCode : int, startDate: str) -> pd.DataFrame :
    ''' 
    ex) startData : "2022-07-11"
    '''
    from datetime import timedelta, datetime
    day = datetime.strptime(startDate,'%Y-%m-%d').date()

    #해당 달의 첫째날 구하기
    first_day = day.replace(day=1)

    

    #전달의 마지막 날 구하기
    dayLast = first_day - timedelta(days=1)
    print(dayLast)

    #전달의 첫째 날 구하기
    dayStart = dayLast.replace(day=1)
    print(dayStart)

    libUrl=f'http://data4library.kr/api/itemSrch?libCode={libCode}&startDt={dayStart}&endDt={dayLast}&authKey=7123eacb2744a02faca2508a82304c3bf154bf0b285da35c2faa2b8498b09872&pageSize=10000'
    libHtml = requests.get(libUrl)
    libSoup = BeautifulSoup(libHtml.content, features="xml")


    libBookname = list(map(lambda x : x.string,libSoup.find_all('bookname')))
    libAuthor =  list(map(lambda x : x.string,libSoup.find_all('authors'))) 
    libPublisher = list(map(lambda x : x.string,libSoup.find_all('publisher')))  
    libISBN = list(map(lambda x : x.string,libSoup.find_all('isbn13')))  
    libClassNum = list(map(lambda x : x.string,libSoup.find_all('class_no')))  
    libRegDate = list(map(lambda x : x.string,libSoup.find_all('reg_date')))  
    libBookImageURL = list(map(lambda x : x.string,libSoup.find_all('bookImageURL')))  

    data = pd.DataFrame([libBookname,libAuthor,libPublisher,libISBN,libClassNum,libRegDate,libBookImageURL]).T
    data.columns = ['도서명', '저자', '출판사', 'ISBN', '주제분류번호', '등록일자','이미지주소']
    sortedData : pd.DataFrame = data[~data['주제분류번호'].isna()]
    return sortedData

In [112]:
k = loadLibBook(lib1,'2022-07-15')

2022-06-30
2022-06-01


In [143]:
bookInfoSearch['keyword'] = result['keyword'].tolist()

In [ ]:
libInfo = pd.read_csv('./data/lib_info.csv',encoding='CP949')

libInfo[libInfo['libName'].str.contains('서울특')]['libCode'].tolist()

In [ ]:
# 날짜만 넣으면 keybert 결과까지 나올 수 있게 정리하기

# 서울 교육청 도서관만 넣기
def extractAllLibBooks(startDate:str) -> pd.DataFrame :
    libCode = [111003, 111004, 111005, 111006, 111007, 111008, 111009, 111010, 111022, 111011, 111012, 111013, 111014, 111031, 111016, 111017, 111030, 111015, 111018, 111019, 111020,
    111021]
    dataframes = []
    for code in tqdm(libCode[:2]) : 
        libBook :pd.DataFrame = loadLibBook(code,startDate)  
        dataframes.append(libBook)

    result= pd.concat(dataframes).drop_duplicates(subset='ISBN')
    val = result['주제분류번호'].astype(str)
    BM = val.str.contains('005') | val.str.contains('004')
    result : pd.DataFrame = result[BM]
    return result

asd = extractAllLibBooks('2022-07-15')

asd



39